In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC # Logistic Regression - Credit Default Prediction

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 1: Load Libraries and Data

# COMMAND ----------

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix,
    classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

print("=== Starting Logistic Regression Model ===\n")

# Load the scaled data
spark.sql("USE credit_risk")
df_spark = spark.table("application_train_scaled")

print(f"Loaded data: {df_spark.count():,} rows × {len(df_spark.columns)} columns")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 2: Convert to Pandas

# COMMAND ----------

print("Converting to Pandas (this may take 2-3 minutes)...")
df = df_spark.toPandas()

print(f"✅ Converted to Pandas: {df.shape[0]:,} rows × {df.shape[1]} columns")
print()

# Check for any missing values
missing_count = df.isnull().sum().sum()
print(f"Missing values: {missing_count}")

if missing_count > 0:
    print("⚠️ WARNING: Missing values detected!")
else:
    print("✅ No missing values")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 3: Prepare Features and Target

# COMMAND ----------

# Separate features (X) and target (y)
# Remove ID column and target from features

# Identify columns to exclude from features
exclude_cols = ['SK_ID_CURR', 'TARGET']

# Get feature columns
feature_cols = [col for col in df.columns if col not in exclude_cols]

# Create feature matrix (X) and target vector (y)
X = df[feature_cols]
y = df['TARGET']

print(f"=== Data Preparation ===")
print(f"Total samples: {len(df):,}")
print(f"Number of features: {len(feature_cols)}")
print(f"\nFeature columns (first 20):")
for i, col in enumerate(feature_cols[:20], 1):
    print(f"  {i}. {col}")
if len(feature_cols) > 20:
    print(f"  ... and {len(feature_cols) - 20} more")

print(f"\nTarget variable distribution:")
print(y.value_counts())
print(f"\nClass balance:")
print(f"  No Default (0): {(y == 0).sum():,} ({(y == 0).mean()*100:.2f}%)")
print(f"  Default (1):    {(y == 1).sum():,} ({(y == 1).mean()*100:.2f}%)")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 4: Split Data into Train and Test Sets

# COMMAND ----------

# Split: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Maintain class balance in both sets
)

print(f"=== Train/Test Split ===")
print(f"Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Test set:     {X_test.shape[0]:,} samples ({X_test.shape[0]/len(df)*100:.1f}%)")
print(f"\nFeatures: {X_train.shape[1]}")

print(f"\nTraining set class distribution:")
print(f"  No Default (0): {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.2f}%)")
print(f"  Default (1):    {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.2f}%)")

print(f"\nTest set class distribution:")
print(f"  No Default (0): {(y_test == 0).sum():,} ({(y_test == 0).mean()*100:.2f}%)")
print(f"  Default (1):    {(y_test == 1).sum():,} ({(y_test == 1).mean()*100:.2f}%)")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 5: Train Logistic Regression Model

# COMMAND ----------

print("=== Training Logistic Regression Model ===\n")
print("This may take a few minutes with ~100 features and 245K training samples...")

# Initialize logistic regression
# max_iter increased to ensure convergence with many features
logreg = LogisticRegression(
    random_state=42,
    max_iter=1000,
    solver='lbfgs',  # Good for large datasets
    class_weight='balanced'  # Handle class imbalance
)

# Train the model
print("Training...")
logreg.fit(X_train, y_train)

print("✅ Model training complete!")
print(f"\nModel converged: {logreg.n_iter_[0]} iterations")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 6: Make Predictions

# COMMAND ----------

print("=== Making Predictions ===\n")

# Predict on training set
y_train_pred = logreg.predict(X_train)
y_train_pred_proba = logreg.predict_proba(X_train)[:, 1]

# Predict on test set
y_test_pred = logreg.predict(X_test)
y_test_pred_proba = logreg.predict_proba(X_test)[:, 1]

print("✅ Predictions complete")
print(f"Training predictions: {len(y_train_pred):,}")
print(f"Test predictions: {len(y_test_pred):,}")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 7: Evaluate Model Performance

# COMMAND ----------

print("="*60)
print("MODEL PERFORMANCE EVALUATION")
print("="*60)

# Training set performance
print("\n📊 TRAINING SET PERFORMANCE:")
print(f"  Accuracy:  {accuracy_score(y_train, y_train_pred):.4f}")
print(f"  Precision: {precision_score(y_train, y_train_pred):.4f}")
print(f"  Recall:    {recall_score(y_train, y_train_pred):.4f}")
print(f"  F1 Score:  {f1_score(y_train, y_train_pred):.4f}")
print(f"  ROC-AUC:   {roc_auc_score(y_train, y_train_pred_proba):.4f}")

# Test set performance
print("\n📊 TEST SET PERFORMANCE:")
print(f"  Accuracy:  {accuracy_score(y_test, y_test_pred):.4f}")
print(f"  Precision: {precision_score(y_test, y_test_pred):.4f}")
print(f"  Recall:    {recall_score(y_test, y_test_pred):.4f}")
print(f"  F1 Score:  {f1_score(y_test, y_test_pred):.4f}")
print(f"  ROC-AUC:   {roc_auc_score(y_test, y_test_pred_proba):.4f}")

# COMMAND ----------

# MAGIC %md
# MAGIC ### Metric Interpretations
# MAGIC 
# MAGIC - **Accuracy**: Overall correctness - what % of predictions were correct
# MAGIC - **Precision**: Of predicted defaults, what % actually defaulted (avoid false alarms)
# MAGIC - **Recall**: Of actual defaults, what % did we catch (avoid missing defaults)
# MAGIC - **F1 Score**: Balance between precision and recall
# MAGIC - **ROC-AUC**: Overall model quality (0.5 = random, 1.0 = perfect)

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 8: Confusion Matrix

# COMMAND ----------

# Calculate confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

print("\n=== CONFUSION MATRIX (Test Set) ===\n")
print("                  Predicted")
print("                No Default  Default")
print(f"Actual No Default   {cm[0,0]:6,}    {cm[0,1]:6,}")
print(f"Actual Default      {cm[1,0]:6,}    {cm[1,1]:6,}")

print("\nBreakdown:")
print(f"  True Negatives (TN):  {cm[0,0]:,} - Correctly predicted no default")
print(f"  False Positives (FP): {cm[0,1]:,} - Incorrectly predicted default")
print(f"  False Negatives (FN): {cm[1,0]:,} - Missed actual defaults ⚠️")
print(f"  True Positives (TP):  {cm[1,1]:,} - Correctly predicted default")

# COMMAND ----------

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Default', 'Default'],
            yticklabels=['No Default', 'Default'])
plt.title('Confusion Matrix - Test Set', fontsize=16, fontweight='bold')
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 9: ROC Curve

# COMMAND ----------

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)
roc_auc = roc_auc_score(y_test, y_test_pred_proba)

# Plot ROC curve
plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier (AUC = 0.50)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=16, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📈 ROC-AUC Score: {roc_auc:.4f}")
print("\nInterpretation:")
print("  • 0.50 = Random guessing (no predictive power)")
print("  • 0.70 = Acceptable model")
print("  • 0.80 = Good model")
print("  • 0.90 = Excellent model")
print("  • 1.00 = Perfect model")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 10: Classification Report

# COMMAND ----------

print("\n=== DETAILED CLASSIFICATION REPORT (Test Set) ===\n")
print(classification_report(y_test, y_test_pred, 
                          target_names=['No Default (0)', 'Default (1)'],
                          digits=4))

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 11: Feature Importance Analysis

# COMMAND ----------

print("=== FEATURE IMPORTANCE ===\n")
print("Analyzing which features are most predictive of default...\n")

# Get coefficients
coefficients = logreg.coef_[0]

# Create dataframe of feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
})

# Sort by absolute value of coefficient (impact regardless of direction)
feature_importance = feature_importance.sort_values('abs_coefficient', ascending=False)

# Top 20 most important features
print("📊 TOP 20 MOST IMPORTANT FEATURES:\n")
top_20 = feature_importance.head(20)

for i, row in enumerate(top_20.itertuples(), 1):
    direction = "↑ Increases" if row.coefficient > 0 else "↓ Decreases"
    print(f"{i:2}. {row.feature:40} | Coef: {row.coefficient:8.4f} | {direction} default risk")

# COMMAND ----------

# Visualize top 20 features
plt.figure(figsize=(12, 8))
top_20_plot = feature_importance.head(20).sort_values('coefficient')
colors = ['red' if x < 0 else 'green' for x in top_20_plot['coefficient']]

plt.barh(range(len(top_20_plot)), top_20_plot['coefficient'], color=colors, alpha=0.7)
plt.yticks(range(len(top_20_plot)), top_20_plot['feature'])
plt.xlabel('Coefficient Value', fontsize=12)
plt.title('Top 20 Most Important Features\n(Red = Decreases Default Risk, Green = Increases Default Risk)', 
          fontsize=14, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 12: Prediction Probability Distribution

# COMMAND ----------

# Plot distribution of predicted probabilities
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(y_test_pred_proba[y_test == 0], bins=50, alpha=0.7, label='Actual No Default', color='blue')
plt.hist(y_test_pred_proba[y_test == 1], bins=50, alpha=0.7, label='Actual Default', color='red')
plt.xlabel('Predicted Probability of Default', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.title('Distribution of Predicted Probabilities', fontsize=13, fontweight='bold')
plt.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Decision Threshold (0.5)')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot([y_test_pred_proba[y_test == 0], y_test_pred_proba[y_test == 1]], 
            labels=['Actual No Default', 'Actual Default'])
plt.ylabel('Predicted Probability of Default', fontsize=11)
plt.title('Predicted Probability by Actual Class', fontsize=13, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nGood separation means:")
print("  • Blue (no default) concentrated near 0")
print("  • Red (default) concentrated near 1")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 13: Sample Predictions

# COMMAND ----------

print("=== SAMPLE PREDICTIONS ===\n")

# Create sample predictions dataframe
sample_predictions = pd.DataFrame({
    'SK_ID_CURR': df.loc[X_test.index, 'SK_ID_CURR'].values,
    'Actual': y_test.values,
    'Predicted': y_test_pred,
    'Probability_Default': y_test_pred_proba
})

# Show some correct predictions
print("✅ Correctly Predicted Defaults (first 10):")
correct_defaults = sample_predictions[(sample_predictions['Actual'] == 1) & 
                                     (sample_predictions['Predicted'] == 1)].head(10)
print(correct_defaults.to_string(index=False))

print("\n✅ Correctly Predicted No Defaults (first 10):")
correct_no_defaults = sample_predictions[(sample_predictions['Actual'] == 0) & 
                                         (sample_predictions['Predicted'] == 0)].head(10)
print(correct_no_defaults.to_string(index=False))

print("\n❌ Incorrectly Predicted (False Positives - first 10):")
false_positives = sample_predictions[(sample_predictions['Actual'] == 0) & 
                                    (sample_predictions['Predicted'] == 1)].head(10)
print(false_positives.to_string(index=False))

print("\n⚠️ Missed Defaults (False Negatives - first 10):")
false_negatives = sample_predictions[(sample_predictions['Actual'] == 1) & 
                                    (sample_predictions['Predicted'] == 0)].head(10)
print(false_negatives.to_string(index=False))

# COMMAND ----------

# COMMAND ----------
# MAGIC %md
# MAGIC ## Step 14: Save Results (Skip Model Saving on Serverless)

# COMMAND ----------

print("=== Saving Results ===\n")

# Save feature importance to table instead
feature_importance_spark = spark.createDataFrame(feature_importance)
feature_importance_spark.write.mode("overwrite").saveAsTable("credit_risk.feature_importance")

print("✅ Feature importance saved to table: credit_risk.feature_importance")

# Save test predictions to table
sample_predictions_spark = spark.createDataFrame(sample_predictions)
sample_predictions_spark.write.mode("overwrite").saveAsTable("credit_risk.test_predictions")

print("✅ Test predictions saved to table: credit_risk.test_predictions")

print("\n⚠️ Note: Model not saved (serverless compute limitation)")
print("   You can retrain the model anytime by running this notebook again")

# COMMAND ----------

# MAGIC %md
# MAGIC ## Step 15: Final Summary

# COMMAND ----------

print("\n" + "="*70)
print("LOGISTIC REGRESSION MODEL - FINAL SUMMARY")
print("="*70)

print(f"\n📊 Dataset:")
print(f"  • Total samples: {len(df):,}")
print(f"  • Training samples: {len(X_train):,} (80%)")
print(f"  • Test samples: {len(X_test):,} (20%)")
print(f"  • Number of features: {len(feature_cols)}")

print(f"\n🎯 Model Performance (Test Set):")
print(f"  • Accuracy:  {accuracy_score(y_test, y_test_pred):.4f}")
print(f"  • Precision: {precision_score(y_test, y_test_pred):.4f}")
print(f"  • Recall:    {recall_score(y_test, y_test_pred):.4f}")
print(f"  • F1 Score:  {f1_score(y_test, y_test_pred):.4f}")
print(f"  • ROC-AUC:   {roc_auc_score(y_test, y_test_pred_proba):.4f}")

print(f"\n🔍 Confusion Matrix (Test Set):")
print(f"  • True Negatives:  {cm[0,0]:,}")
print(f"  • False Positives: {cm[0,1]:,}")
print(f"  • False Negatives: {cm[1,0]:,}")
print(f"  • True Positives:  {cm[1,1]:,}")

print(f"\n⭐ Top 3 Most Important Features:")
for i, row in enumerate(feature_importance.head(3).itertuples(), 1):
    print(f"  {i}. {row.feature} (coefficient: {row.coefficient:.4f})")

print(f"\n💾 Saved Files:")
print(f"  • Model: {model_path}")

print("\n" + "="*70)
print("🎉 LOGISTIC REGRESSION COMPLETE!")
print("="*70)

# COMMAND ----------